In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
! pip install datasets transformers[sentencepiece] rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 8.0 MB/s 
     |████████████████████████████████| 4.4 MB 77.2 MB/s 
     |████████████████████████████████| 140 kB 94.9 MB/s 
     |████████████████████████████████| 212 kB 98.3 MB/s 
     |████████████████████████████████| 1.1 MB 72.9 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 596 kB 94.6 MB/s 
     |████████████████████████████████| 127 kB 91.9 MB/s 
     |████████████████████████████████| 144 kB 75.8 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 271 kB 78.0 MB/s 
     |████████████████████████████████| 6.6 MB 42.0 MB/s 
     |████████████████████████████████| 1.2 MB 75.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib

In [ ]:
import transformers
import datasets
import random
import pandas as pd
import numpy as np
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir = '/content/drive/MyDrive/adl-hw3/data/train.jsonl'
sample_dir = '/content/drive/MyDrive/adl-hw3/data/public.jsonl'
train_data = [json.loads(line) for line in open(train_dir, 'r')]
valid_data = [json.loads(line) for line in open(sample_dir, 'r')]
print(len(train_data))
print(len(valid_data))

21710
5494


In [ ]:
document = []
summary = []
id = []
for train in train_data:
  document.append(train['maintext'])
  summary.append(train['title'])
  id.append(train['id'])

train = dict({
    'document': document,
    'summary': summary,
    'id': id,
})

document = []
summary = []
id = []
for valid in valid_data:
  document.append(valid['maintext'])
  summary.append(valid['title'])
  id.append(valid['id'])

valid = dict({
    'document': document,
    'summary': summary,
    'id': id,
})

from datasets import Dataset
train_dataset = Dataset.from_dict(train)
valid_dataset = Dataset.from_dict(valid)
train_dataset
valid_dataset

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 5494
})

# Preprocessing

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

In [ ]:
from transformers import MT5Model, T5Tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
#tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = ['summarize: ' + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(train_dataset[:2])

{'input_ids': [[196098, 10701, 267, 259, 71934, 2144, 3981, 10559, 17481, 25456, 10815, 493, 12258, 241245, 178273, 261, 259, 1083, 137179, 1193, 218588, 218588, 5039, 188711, 2446, 261, 4002, 74378, 9734, 2904, 47163, 22399, 4484, 98942, 1146, 91653, 31225, 9027, 261, 259, 7609, 40439, 51104, 57089, 63171, 26387, 2910, 4565, 493, 15862, 8369, 306, 170591, 213589, 26539, 1374, 13148, 37222, 3541, 7605, 261, 11250, 56656, 4153, 37222, 77478, 4066, 23281, 38706, 145248, 2910, 4565, 86305, 306, 259, 71934, 2144, 120257, 8813, 27474, 45583, 5991, 29693, 493, 11250, 261, 116772, 63687, 18540, 137179, 1193, 175062, 23823, 4457, 12307, 11072, 261, 259, 8779, 16332, 170016, 16436, 26387, 2910, 4565, 1637, 70375, 64549, 493, 175301, 261, 14783, 8451, 33692, 219191, 241181, 5070, 306, 259, 3178, 16577, 1374, 13591, 848, 13591, 891, 15778, 31225, 838, 261, 98942, 187717, 29693, 154194, 170591, 70654, 9139, 31225, 8553, 51061, 21270, 48137, 8553, 127850, 156415, 21270, 5507, 848, 137357, 493, 1593

In [ ]:
train_datasets = train_dataset.map(preprocess_function, batched=True)
valid_datasets = valid_dataset.map(preprocess_function, batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
print(train_datasets[0])

{'document': '從小就很會念書的李悅寧， 在眾人殷殷期盼下，以榜首之姿進入臺大醫學院， 但始終忘不了對天文的熱情。大學四年級一場遠行後，她決心遠赴法國攻讀天文博士。 從小沒想過當老師的她，再度跌破眾人眼鏡返台任教， 只為幫助對天文有興趣的學生，少走點冤枉路。\n二○一九年九月開學日，臺灣師範大學地球科學系迎來創系三十五年來最年輕的助理教授、還不滿三十歲的李悅寧。 時間往前推半年多，科技部部長陳良基率團到歐洲，延攬海外優秀學術科研人才返國貢獻。在巴黎那場，科技部二○一九年「愛因斯坦培植計畫」得主之一的李悅寧，親自向陳良基報到，表達她打算返台，回臺師大任職，投入台灣天文研究行列的動向。\n這距離她到法國當交換學生，之後從臺大醫學系休學，轉換跑道留在法國念天文，整整七年。\n高中展露科學天分 卻務實選擇醫學系\n李悅寧就讀台中女中高三時期，身邊的同學大多忙著準備大學指考，她卻因緣際會接觸到地球科學，在老師鼓勵下參加首屆國際地球科學奧林匹亞競賽，沒想到一鳴驚人，抱回金牌及全球第二名。 在一片「聰明的人念醫學院，最厲害的人念臺大醫科」的氛圍中，二○○八年，李悅寧頂著當年大學指考第二、 三類組榜首的光環，很自然地在眾人期盼下進入臺大醫學院。 李悅寧坦言，儘管當時對地球科學及研究領域很有興趣，但「有點沒安全感」，想做點「安全、務實的事」；相形之下，醫學系所讀的知識不但務實，也都是之前沒學過的內容。她甚至想好了醫學院畢業之後，要往創傷重建的整形外科方向走。\n雖然進入醫學系就讀，李悅寧卻始終無法忘懷科學研究。除了從大二開始雙主修醫學系及機械工程系，每個暑假她幾乎都投身科學研究相關活動。 二○○九年及二○一○年暑假，在老師推薦與科技部前身國科會的經費支持下，李悅寧分別到法國巴黎出席國際天文年的開幕儀式、赴德國參加林島諾貝爾獎得主會議（Lindau Nobel Laureate Meeting），並自己找資訊，申請前往瑞士洛桑聯邦理工學院（EPFL），參加心理物理學相關活動。 大四升大五的暑假，李悅寧到中研院天文及天文物理研究所，和專長恆星形成及天文化學的副研究員呂聖元合作進行專題計畫，當時她就萌生轉換跑道的念頭，卻一直無法決定，乾脆利用一年時間，為自己安排一場學習之旅。\n赴法交換學生 改變人生發展\n一般來說，醫學系的交換學生多是到醫院實習，比較少有校級常規交換生，不過

In [ ]:
tokenizer.decode(train_datasets[0]['labels'])

'榜首進台大醫科卻休學 、27歲拿到法國天文博士 李悅寧跌破眾人眼鏡返台任教</s>'

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

In [ ]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    output_dir = "/content/drive/MyDrive/adl-hw3/result",
    evaluation_strategy = "epoch",
    save_strategy="steps",
    save_steps= 27140,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    gradient_accumulation_steps = 2,
    save_total_limit = 1,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import nltk
import numpy as np

nltk.download('punkt')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_datasets,
    eval_dataset=valid_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
!nvidia-smi

Sat Jul  2 08:53:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: id, document, summary. If id, document, summary are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21710
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 27140


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,5.252100,3.956136,9.371800,3.657300,9.332200,9.347700,12.871500
2,4.540300,3.792492,11.098100,4.229500,11.028900,11.042200,15.111900
3,4.305600,3.670285,11.823800,4.548100,11.698300,11.692800,17.077900
4,4.182400,3.612419,12.394800,4.517000,12.276600,12.239900,17.094300
5,4.070100,3.547849,12.834200,4.569300,12.692300,12.693600,17.379500
6,3.978000,3.515932,13.318700,4.709900,13.183000,13.172700,17.436700
7,3.941800,3.484325,13.432600,4.846900,13.322900,13.328900,17.625400
8,3.887700,3.462155,13.639000,4.912900,13.494900,13.497000,17.551500
9,3.847100,3.431443,14.447200,5.050300,14.297700,14.323500,17.774100
10,3.809900,3.428612,13.952600,4.880600,13.798000,13.823200,17.715100


The following columns in the evaluation set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: id, document, summary. If id, document, summary are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5494
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: id, document, summary. If id, document, summary are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5494
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: id, document, summary. If id, document, summary are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this me

TrainOutput(global_step=27140, training_loss=4.015673938321118, metrics={'train_runtime': 18676.974, 'train_samples_per_second': 23.248, 'train_steps_per_second': 1.453, 'total_flos': 2.29583115780096e+17, 'train_loss': 4.015673938321118, 'epoch': 20.0})

# Testing

In [ ]:
! pip install datasets transformers[sentencepiece] rouge-score nltk

     |████████████████████████████████| 325 kB 7.6 MB/s 
     |████████████████████████████████| 4.0 MB 79.7 MB/s 
     |████████████████████████████████| 1.1 MB 77.1 MB/s 
     |████████████████████████████████| 212 kB 79.0 MB/s 
     |████████████████████████████████| 77 kB 7.9 MB/s 
     |████████████████████████████████| 136 kB 83.7 MB/s 
     |████████████████████████████████| 127 kB 98.2 MB/s 
     |████████████████████████████████| 144 kB 87.4 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 90.7 MB/s 
     |████████████████████████████████| 880 kB 89.0 MB/s 
     |████████████████████████████████| 6.6 MB 64.0 MB/s 
     |████████████████████████████████| 596 kB 77.6 MB/s 
     |████████████████████████████████| 1.2 MB 75.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=77bf60e099d62422c5bb82e76bba0f6804ab04c89be1b9426873912e54945c02
  Stored in directory: /root/.cac

In [ ]:
!git clone https://github.com/moooooser999/ADL22-HW3.git
%cd ADL22-HW3
!pip install -e tw_rouge

Cloning into 'ADL22-HW3'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 34 (delta 7), reused 13 (delta 1), pack-reused 0
Unpacking objects: 100% (34/34), done.
/content/ADL22-HW3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/ADL22-HW3/tw_rouge
  Running setup.py develop for tw-rouge


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_dir = '/content/drive/MyDrive/adl-hw3/result/checkpoint-27140'

Mounted at /content/drive


In [ ]:
import json
import tqdm
import numpy as np
import random
from transformers import AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, T5TokenizerFast, MT5ForConditionalGeneration
#from tw_rouge import get_rouge
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
!nvidia-smi

Sat Jul  2 09:43:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class T5_Dataset(Dataset):
    def __init__(self, data):
      self.ids = data['input_ids']
      self.title = data['labels']
      self.sum = data['summary']
    def __getitem__(self, idx):
        return torch.as_tensor(self.ids[idx]), self.sum[idx]
    
    def __len__(self):
        return len(self.ids)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

In [ ]:
sample_dir = '/content/drive/MyDrive/adl-hw3/data/public.jsonl'
valid_data = [json.loads(line) for line in open(sample_dir, 'r')]
print(len(valid_data))

5494


In [ ]:
document = []
summary = []
id = []
for valid in valid_data:
  document.append(valid['maintext'])
  summary.append(valid['title'])
  id.append(valid['id'])

valid = dict({
    'document': document,
    'summary': summary,
    'id': id,
})

from datasets import Dataset
valid_dataset = Dataset.from_dict(valid)
valid_dataset

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 5494
})

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = ['summarize: ' + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
valid_datasets = valid_dataset.map(preprocess_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
dev_dataset = T5_Dataset(valid_datasets)
dev_dataloader = DataLoader(dev_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [ ]:
!ls

eval.py  README.md  tw_rouge


In [ ]:
#from tw_rouge import get_rouge
import tqdm
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)
res = []
title = []
id = []
batch = 32
model.eval()
'''
greedy
num_beams= 5 or 10
top_k = 10 or 20
top_p = 0.92 or 0.7
temperature = 0.5 or 2
'''
for p, q in tqdm.tqdm(dev_dataloader):
    p = p.to(device)
    output = tokenizer.batch_decode(model.generate(p, temperature=0.5, max_length=128, repetition_penalty=3.0, do_sample=True),skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(output)
    res.extend(output)
    id.extend()
#rouge = get_rouge(res, title)
#print(rouge)

cuda:0


  1%|          | 1/172 [00:01<02:57,  1.04s/it]

['Anker推出新款真無線藍牙耳機Liberty Air 2 Pro', '鐵道自行車必吃!苗栗「三義舊山線鐵路騎行」、全台最熱門路線', '華碩推出換上Intel第11代Core處理器的Chromebook Flip CX5', '科研院產學所長蘇孟宗:疫情改變產業發展', '全球有多少裝置不運行Windows 7?微軟公布「數字至少超過 1億台」', '台幣回流價位破兆元!', '馬克杯上架後爆銷1萬件!網友留言「我不只治療」、亞太遜買賣缺貨', '華碩更新雙螢幕筆電ZenBook Duo 14、Galaxy Pro Duo 15OLED', '吃「雞」加酒!臺虎推週末炸猴俱樂部,超夯美食', '哈登、火箭鬧劇落幕!紐媒爆料厄文未曝被獲知有關狀況的小道消息', '健身教練:運動讓生活更好', '華碩推出ZenBeam Latte小型LED投影機', '當一個不願柔軟的人?讓我變成個無法獨秀的權威者', '華碩推出隸屬ProArt顯示器系列 加入實體滾輪控制', '自己有適合人?該如何找到「個性」為何不適的學習法', '車聯網、智慧交通產業政策戰略重心', '金士頓首款PCIe Gen 4.0 SSD新品推出', '疫情「慘」業!台灣塑化股暴跌逾20%', '晶圓代工龍頭台積電法說會:上半年業績季增1.4%、年增22%', '不平等至關重要:讓讀者反思為什麼我們有權利且應該追求公平', '為何H&M選年度最經典聯名系列?', '星巴克「巨無霸杯」超吸睛!台北101購物中心限定開賣「500馬克咖啡杯」、推20倍', '鴻海集團衝刺數位化、F2.0數字富士康啟動燈塔工廠', '不婚主義到即將迎接新生命!廖曉喬揭開「媽母」', '全球資金回流、台商股受影響', '小米11發熱問題?全球評測者發現「全台積電」的手機是什麼原因', '不斷開雜亂人生!3歲母親離家生活', '台灣設計界出爐!2020年度APDC亞太建築精英邀請賽得獎名單全揭曉', '疫情變化!2021台北國際書展改以線上書店辦理', '華碩正式揭曉雙螢幕筆電ZenBook Duo 14', '醫療保障卻不足!AI友邦人壽:未來健保只提供基礎的治療服務', '不平等至關重要?為何「稅收」到當重大的']


  1%|          | 2/172 [00:01<02:47,  1.01it/s]

['孕媽咪要開啟「荷爾蒙」!愛無旁貸地成為最操盤手', '《電馭叛客 2077》將延期至 2022年', '婚姻的愛情故事/婚前談錢不可愛?', '台積電「護國神山」一海比二山高!聯發科股價逼近千元', '台積電為何在半導體需求?外媒:美市場對台灣一事有影了', 'Converse打造新年新希望!「極致灰」代表色在鞋款中', '火箭快刀斬亂麻送走哈登!兩隊交易放棄三巨頭', '英特爾專利權大戰失業?兩項是美史上罰款最重', '華碩推出商務筆電ExpertBook B9400CEAV', '孕後陰道鬆弛、漏尿危機?', '塔羅:2021工作運勢如何?', '陳進財:堅持走一條難走的路', '彰化「安心即時上工計畫」釋出680個工作機會', '設計規劃:空間的清新氛圍', '員林蜜餞王國凋零!台灣傳統甜食市場卻7年增6倍', '疫情上熱議排行榜十大人氣保健品牌!2020年新希望就從健康開始', 'DHL國際快遞續購八架新型波音777貨機', '台灣光環境獎Light Art 2020台影科技專家', '醫師:孕期不適關鍵字,10個胎兒有溫暖', '露天「賣家發財金」貸款1.111%數字乍看下對買家的來講非常友善', 'MLB/克魯柏控球依舊神準 但僅剩88-90英里', '不被拋棄而持續衝出教室?大鳩信賴/自覺是一種更富社交性', '台積電投資發財開啟!萬寶投顧蔡明彰:散戶不要當市場的小紅帽', '外貿協會首度辦理2020年度記者會 黃志芳:未來將加速數位轉型', 'MLB/火箭把陣中球星賣我?洗車店讓顧客用哈登換免費洗衣服務', '疫情影響口罩生產 Razer推出概念產品設計', '愛馬仕「B. Blossom」經典珠寶圖案!華麗風格、紅色蛋白石的巴黎情人節送禮', '楓紅「HANA、BIYORI」最美時機!日本庭園櫻花風光與紅葉燈籠點上', '台股高檔震盪,外資點火:單日買超35.8億元', '星巴克門市、森林咖啡大區!新北淡水雲門「天玉」', '火箭改朝換代', '年輕人瘋買股票?這些要點也不能繳學費']


...

['跑步經濟性要如何增加?', '訓練心得/如何計算80%的跑步量?', '研華第1季營收創歷史次高', '不動產投資信託基金「新光一號」遭市場派狙擊!受託銀行拒投票', '碩天科技新在線互動式 PFC正弦波不斷電系統 CP1500PFCLCDa獲美國 The Channel Company旗下雜誌 CRN年度產品獎', '中歐關系陷入低谷?媒:雙方不希望對話繼續下行', '三星攜五位新銳導演接力拍片!雙星推「旗艦級手機攝影」', '日立收購美國數位工程服務領導廠商GlobalLogic', '金士頓攜恩智浦合作打造全新i.MX 8M Plus應用處理器', '三星準備與Olympus攜手合作 Galaxy Z Fold 3', '美國政府預計明年年中達成全球性且以共識為基礎的解決方案', '美國YouTuber擅闖德州 SpaceX Starship車', '女將於懷胎8個月下奪冠軍!男會批評:不該冒險進行比賽', '人機介面將成為企業數位轉型', '被「蕉流」男優不愛乾淨的女權力?', '宏碁舉辦Predator League虛擬賽車聯賽', '美國商務部將7間業者、機構列入禁止清單', '宏碁首屆Predator League虛擬賽車聯賽 發揮職業、科技產業', '美國帳戶超過3000萬人!Facebook爆發用資外洩事件', '機器視覺技術跨界發展', '全台5家「杯子蛋糕」店!新北市板橋區超夯品牌,還有傳統咖啡館', '在手心啜飲的食譜!生薑入茶「餐品」', '與澎湖縣政府合作 LINE攜手「2020」活動', '塔羅:我顧人怨指數?', '歐洲人權法院裁定:強制接種疫苗不違反法律', '特斯拉調漲美國Model 3、 Model Y售價', '中國大陸超級電腦企業列入實體清單?陸媒點名「中槍」被喊冤', '工研院5G技術開台後!設計與通訊研究', 'LINE推出2020澎湖國際海上花火節 打造智慧旅遊體驗', '駭客入侵佛羅里達州坦帕市供水處理設施', '佐藤可士和正式起跑!日本設計大師「SWAT、Kashiwa Sato」開賣', '嫩奶油、牛乳汁味!俏媽咪示範:美味的肉燥和甜蜜']


 99%|█████████▉| 171/172 [02:54<00:01,  1.07s/it]

['疫情後、矽谷買氣也變旺跡象?', '黃姓男子不成立個資法案!法院判處有期徒刑3個月', '科技的力量×社會新創不打烊研討會', '台鐵事故真相徹底檢討!', '三星推出藍牙定位配件Galaxy SmartTag+!一月攜手Xperia S21系列手機打造UWB技術規格的新款', '歐美風格!時尚品牌Aimé Leon Dore聯名帆船鞋', '大稻埕碼頭貨櫃市集4月9日盛大開幕!全新「清水河岸」新藝象', '英國菲立普親王逝世終年99歲', 'MLB/天使奪冠加州球飛出場外!大聯盟3局上轟5戰', '小安:強姦者被家暴 「這件事」遭家人產生矛盾', 'MLB/無法再見觸身球?馬林魚:他不挨接擊敗', 'Thunderbolt應用問題如何解決USB 4?', 'MLB/米尼蘇達雙城首戰勝利!4局重返主場作賽', '美股大崩壞或跌4成!全球有「最大風險」', '美國每部 iPhone平均支出比前一年增加38%', '理財、理善最重要代表', '智慧手機用戶對5G不十分感興趣?', '韓國女星河智苑以畫家身份出道!「Converse」鞋款曝光', '印尼攜手印度交涉 獲多達1億支中國新冠疫苗', '全明星光環仍在!金布洛爾奪生涯第350場救援成功', '搶攻母親節檔期!MM6Maison MargielaWhite Space快閃店登場,限定7折起', '自由系統結盟後建立深層策略合作關係 與研華共同打造IoT智慧應用場域', '白富美/黃采緹:你以為的黑姑娘奮鬥史', '智慧科技應用開發商機!iPhone 12的成本高出21%', '網友:別問我老公?', '網友揭露Facebook個資外洩事件被竊取 5.3億筆資料', '智慧醫療應用與發展 TDUA:全球最大顯示科技產業', '高級鐘表盛會登場!HUBLOT鮮色計時碼表太誘人', '買零股領到紀念品?投資人怎麼賺', '巴黎線上發表 LEV Khesin塑現七彩霓虹般絢爛男裝', '美元股走低!原油收盤進入三周', '研華投資自由系統20%股權結盟']


100%|██████████| 172/172 [02:55<00:00,  1.02s/it]

['總裁給央行同仁的一封公開信', '台鐵太魯閣出軌事件記者會:全國工地8成沒有專任', '光電產業終止衰退 台灣產值1兆4,267億微幅跌0.4%', '台灣三星「發現嘻哈台」快閃體驗會', '看日本長跑文化!來自澳洲的「全馬接力賽」', 'MLB/包爾:防範舞弊', '迷客夏全新推6款「咀嚼系、特調味」!5大手搖控夏季新品全單9折,加碼LINE好友送1元多2杯', '訓練心得/足底筋膜炎:如何舒緩、治療?', '美國發言人:取消對台交往限制', '屏東假車禍真擄人命案曝光!55歲新男被疑遭拒抓捕', '陸軍自願役中尉排長因個人家庭因素為求免除請假期間職役', 'WHO籲支持台灣加入世界衛生組織', '母親節限定!自來吃大餐、蛋糕必看', '英國菲立普親王:在家是什麼意思?', 'MLB/道奇冠軍隊伍主場開幕戰封紅藍寶石!球迷獻上熱笑致詞爆哭', '人生的隧道效應?你為何貧窮難脫身、孤單的人更容易交到朋友', '中職/前小兄弟再見全壘打!美日籍洋投道恩斯換戲', '國務院發布對台交往新準則 鼓勵與台灣雙方互動', '臺虎精釀「福狼祭」活動登場!六月五、周四下午11點開賣', 'LINE Bank首月開戶數目標 預計4月下旬將獲限量禮', '職場潛規則在職!網友熱議地雷工作10大特徵', '長賜輪事件:兩個名詞環繞著這四件事']


In [ ]:
with open('submission_temp=0_5.jsonl', "w", encoding="utf8") as fp:
  i = 21710
  for r in res:
    json.dump({"id": "{}".format(i), "title": r}, fp, ensure_ascii = False)
    i += 1
    fp.write("\n")

print(i)

27204


In [ ]:
from tw_rouge import get_rouge

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /root/.cache/ckiptagger/data.zip
100%|██████████| 1.88G/1.88G [00:25<00:00, 73.8MB/s]
/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initiali

In [ ]:
!ls
%cd B08902015/ADL22-HW3

B08902015  B08902015.zip  drive  __MACOSX  sample_data
/content/B08902015/ADL22-HW3


In [ ]:
submission = [json.loads(line) for line in open('submission_temp=0_5.jsonl', 'r')]
pred = [x['title'] for x in submission]
label = [x['title'] for x in valid_data]

In [ ]:
print(len(pred), len(label))

5494 5494


In [ ]:
score = get_rouge(pred, label)
print(score)

{'rouge-1': {'r': 0.22386599810600713, 'p': 0.24541050938085301, 'f': 0.22641788216030903}, 'rouge-2': {'r': 0.07450937923623023, 'p': 0.08459306478707461, 'f': 0.07624912488949591}, 'rouge-l': {'r': 0.19149222730155177, 'p': 0.21060589226130394, 'f': 0.1937508365973977}}


In [ ]:
%cd ..
!rm -rf B08902015*

/content


In [ ]:
!wget https://www.dropbox.com/s/nkadou9ykxw217k/B08902015.zip?dl=1 --content-disposition

--2022-05-09 04:12:11--  https://www.dropbox.com/s/nkadou9ykxw217k/B08902015.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/nkadou9ykxw217k/B08902015.zip [following]
--2022-05-09 04:12:11--  https://www.dropbox.com/s/dl/nkadou9ykxw217k/B08902015.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc71b3ba884057011bb841a4f0cf.dl.dropboxusercontent.com/cd/0/get/Bk56hW0nWo7yf4CUbN6M_ws08j5e8Aw--nKxiY_GPuWj97MHzl-C_naXZ67SeLTGK4NdEL2TSkQizJQk_yE8ltRZPKDjjz4wFn-aMB_wNvPzWM08VJMdW9ktPw4Kp-RinQkNbGttPKfmxHo_sQGhZcIRA1LTnZFnJYh-Fogjd2mTgMgyk068YirPxzsb2_HDTXA/file?dl=1# [following]
--2022-05-09 04:12:11--  https://uc71b3ba884057011bb841a4f0cf.dl.dropboxusercontent.com/cd/0/get/Bk56hW0nWo7yf4CUbN6M_ws08j5e8Aw--nKxiY_GPu

In [ ]:
!unzip B08902015.zip

Archive:  B08902015.zip
   creating: B08902015/
  inflating: __MACOSX/._B08902015    
  inflating: B08902015/.DS_Store     
  inflating: __MACOSX/B08902015/._.DS_Store  
  inflating: B08902015/test.py       
  inflating: __MACOSX/B08902015/._test.py  
  inflating: B08902015/run.sh        
  inflating: __MACOSX/B08902015/._run.sh  
  inflating: B08902015/README.md     
  inflating: __MACOSX/B08902015/._README.md  
  inflating: B08902015/train.py      
  inflating: B08902015/download.sh   
  inflating: B08902015/report.pdf    
  inflating: __MACOSX/B08902015/._report.pdf  


In [ ]:
%cd B08902015

/content/B08902015


In [5]:
!ls
%cd /content/drive/MyDrive/adl-hw3/b08902015-hw3
!ls

drive  sample_data
/content/drive/MyDrive/adl-hw3/b08902015-hw3
data  download.sh  README.md  report.pdf  run.sh  test.py  train.py


In [6]:
! chmod 777 download.sh
! chmod 777 run.sh
! ./download.sh
! ./run.sh ./data/public.jsonl submission.jsonl

--2022-07-02 09:44:13--  https://www.dropbox.com/sh/3wxffv37exbkqpv/AABeU8bzMhz_ORsF13lpl_C3a?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/dl/3wxffv37exbkqpv/AABeU8bzMhz_ORsF13lpl_C3a [following]
--2022-07-02 09:44:14--  https://www.dropbox.com/sh/dl/3wxffv37exbkqpv/AABeU8bzMhz_ORsF13lpl_C3a
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc35ceb2a63d78aa66faa444bc95.dl.dropboxusercontent.com/zip_download_get/BLtY9LB0oQATCtw2Eu8xpLBJ4NWKKlv5UBJGNBRnGa9Q0eHjrQzjrN5Mia3s1rUGA6CkoUPAIHXzvlsHU9fqd7qgK3KJ8x3AFKjWgZ8gem473Q?dl=1# [following]
--2022-07-02 09:44:14--  https://uc35ceb2a63d78aa66faa444bc95.dl.dropboxusercontent.com/zip_download_get/BLtY9LB0oQATCtw2Eu8xpLBJ4NWKKlv5UBJGNBRnGa9Q0eHjrQzjrN5Mia3s1rUGA6CkoUPAIHXzv

In [7]:
!ls

checkpoint-27140      data	   README.md   run.sh		 test.py
checkpoint-27140.zip  download.sh  report.pdf  submission.jsonl  train.py
